# Black-Scholes analytic European option pricing

This notebook aims to be a simple tool for pricing european options through the Black-Scholes formula. The code is based on that developed in **Joshi, Mark S. (2008) *C++ Design Patterns and Derivatives Pricing*. New York: Cambridge University Press**, though adapted to Python language.

**Disclaimer: I am not the owner of the piece of code shown below. For more information, refer to the book mentioned before.**

European options are derivatives that, upon payment of a premium, give the holder the right, but not the obligation, to buy or sell the underlying at maturity of the option.

The solution to the Black-Scholes partial differential equation for a European option is: $$c_0=S_0N(d_1)-Ke^{-rT}N(d_2)$$ for the call and $$p_0=Ke^{-rT}N\left(-d_2\right)-S_0N\left(-d_1\right)$$ for the put, where $$d_1=\frac{\ln\left(\frac{S_0}{K}\right)+\left(r+\frac{\sigma^2}{2}\right)T}{\sigma\sqrt{T}}$$ and $$d_2=d_1-\sigma\sqrt{T}$$

Recall that the standard normal probability density function is $$f(x)=\frac{1}{\sqrt{2\pi}}e^{\frac{-x^2}{2}}$$ and the cumulative distribution function is given by $$N(x)=\int_{-\infty}^{x} \frac{1}{\sqrt{2\pi}}e^{\frac{-x^2}{2}}\, dx$$

There are many numerical approximations for this function. In his book, Joshi proposes to use the inverse cumulative normal function via the Beasley-Springer/Moro approximation.

In [1]:
from math import exp, sqrt, log, pi

In [2]:
def normpdf(x):
    return (1.0/sqrt(2*pi)*exp(-0.5*x*x))

In [3]:
def normcdf(x):
    k = 1.0/(1.0 + 0.2316419*x)
    ksum = k*(0.319381530 + k*(-0.356563782 + k*(1.781477937 + k*(-1.821255978 + 1.330274429*k))))
    
    if x >= 0.0:
        return (1.0 - (1.0/sqrt(2*pi))*exp(-0.5*x*x) * ksum)
    elif x < 0.0: 
        return 1.0 - normcdf(-x)

In [4]:
def di(i, Expiry, Strike, Spot, Vol, r):
    return (log(Spot/Strike) + (r + ((-1)**(i-1)*0.5*Vol*Vol)*Expiry)/(Vol*sqrt(Expiry)))

In [5]:
def callPrice(Expiry, Strike, Spot, Vol, r):
    return (Spot * normcdf(di(1, Expiry, Strike, Spot, Vol, r))-Strike*exp(-r*Expiry) * normcdf(di(2, Expiry, Strike, Spot, Vol, r)))

In [6]:
def putPrice(Expiry, Strike, Spot, Vol, r):
    return (-Spot*normcdf(-di(1, Expiry, Strike, Spot, Vol, r))+Spot*exp(-r*Expiry) * normcdf(-di(2, Expiry, Strike, Spot, Vol, r)))

In [7]:
Expiry = 1
Strike = 100
Spot = 100
Volatility = 0.2
r = 0.05

callPrice = callPrice(Expiry, Strike, Spot, Volatility, r)
print(f'Call price: {round(callPrice, 5)}')
putPrice = putPrice(Expiry, Strike, Spot, Volatility, r)
print(f'Put price: {round(putPrice, 5)}')

Call price: 10.45058
Put price: 5.57352
